In [1]:
from flask import Flask,render_template,redirect, url_for,request,flash,session
import pandas as pd
from csv import writer
import pymysql

In [2]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'y_so_secret_tcs'
db = pymysql.connect("localhost","root","","bankSQL", autocommit=True )
conn = db.cursor()
global customerid,accid
customerid=0
accid=0

In [3]:
@app.route('/')
def index():
    return  render_template("index.html")



In [4]:
@app.route('/login',methods=['POST','GET'])
def login():
    error = None
    if request.method == 'POST':
        username=request.form['uname']
        psw=request.form['psw']
        print(" ")
        
        try:
            sql=("""select SSN,Username,Password from userstore where Username=%s and Password=%s""",(username,psw))
            conn.execute(*sql)
            ssn,un,psw,=conn.fetchone()
            session['username'],session['psw'],session['SSN']=un,psw,ssn
            print(session['SSN'])
            if request.form['uname'] != un or request.form['psw'] != psw: 
                error="Invalid Credentials. Please try again."
                return render_template('index.html',error=error)
            else:
                return render_template("dash.html",welcome=session.get('username'))
        except:
            return render_template('index.html',error=error)
            print(" ")
            
                
            
    return render_template('index.html',error=error)
       

In [5]:
                
@app.route('/addncus',methods=['POST','GET'])
def addncus():
     if request.method == 'POST':
        ssnid=request.form['ssn']
        name=request.form['name']
        age=request.form['age']
        addl1=request.form['addressl1']
        addl2=request.form['addressl2']
        state=request.form['state']
        city=request.form['city']
        addl1=addl1+addl2 
        global customerid
        customerid+=1
        print(customerid)
        try:
            tuplev=(ssnid,customerid,name,addl1,state,city,age)
            sql=("""insert into Customer(SSNID,Customerid,Name,Address,State,City,Age) VALUES (%s,%s,%s,%s,%s,%s,%s)""",tuplev)
            conn.execute(*sql)
            return render_template("dash.html",welcome=session.get('username'))
        except:
            error="process coudnt b done"
            return render_template('dash.html',welcome=session.get('username'),error=error)
            print("error")
    
     
       
        
            
        
                    
                    
        
  
            



In [6]:
@app.route('/updatecus',methods=['POST','GET'])
def updatecus():
     if request.method == 'POST':
        try:
            ssn=request.form['ssnid']
            sql=("""select SSNID,Customerid,Name,Address,Age from Customer where SSNID=%s""",(ssn))
            conn.execute(*sql)
            cssid,cusid,coname,coaddress,coage=conn.fetchone()
            name=request.form['name']
            age=request.form['age']
            addl1=request.form['address']
            tuplev=(name,addl1,age,cssid)
            sql=("""update Customer set Name=%s,Address=%s,Age=%s where SSNID=%s""",tuplev)
            conn.execute(*sql)
            return render_template("dash.html",welcome=session.get('username'),val=True,cssid=cssid,cusid=cusid,coname=coname,coaddress=coaddress,coage=coage)
        except:
            error="process coudnt b done"
            return render_template('dash.html',welcome=session.get('username'),val=False)
            print("error")
            
        

In [7]:
@app.route('/searchcus',methods=['POST','GET'])
def searchcus():
     if request.method == 'POST':
        try:
            ssn=request.form['ssnid']
            cid=request.form['cid']
            session['cid'],session['cssn']=cid,ssn
                              
        except:
            error="process coudnt b done"
            return render_template('dash.html',welcome=session.get('username'),val=False)
            print("error")
            
            
            









In [8]:
@app.route('/adeletecus',methods=['POST','GET'])
def adeletecus():
    sql=("""select SSNID,Customerid,Name,Address,Age from Customer where SSNID=%s""",(session.get('cssn')))
    conn.execute(*sql)
    cssid,cusid,cname,cadd,cage=conn.fetchone()
    sql=("""delete from Customer where SSNID=%s and Customerid=%s""",(cssid,cusid))
    session.pop('cssn', None)
    session.pop('cid', None)
    return render_template("dash.html",welcome=session.get('username'),delval=True,cssid=cssid,cusid=cusid,cname=cname,cadd=cadd,cage=cage)






In [9]:
@app.route('/logout',methods=['POST','GET'])
def logout():
    if request.method == 'POST':
        if 'lout' in request.form:
                    session.pop('username', None)
                    session.pop('passwrd', None)
                    session.pop('SSN', None)
                    return render_template("index.html")
        


In [10]:
@app.route('/addaccount',methods=['POST','GET'])
def addaccount():
    if request.method == 'POST':
        cid=request.form['cid']
        acctype=request.form['acctype']
        amt=request.form['amount']
        try:
            global accid
            accid+=1
            tuplev=(accid,cid,amt,acctype)
            sql=("""insert into Account(Accountid,cusid,Balance,accounttype) VALUES (%s,%s,%s,%s)""",tuplev)
            conn.execute(*sql)
            return render_template('dash.html',welcome=session.get('username'))
        except:
            error="process coudnt b done"
            return render_template('dash.html',welcome=session.get('username'),val=False)
            print("error")
        
            
            
            

In [11]:
@app.route('/deleteaccount',methods=['POST','GET'])
def deleteaccount():
    if request.method == 'POST':
        cid=request.form['cid']
        acctype=request.form['acctype']
        aid=request.form['aid']
    
        try:
            tuplev=(aid,cid)
            sql=("""delete from Account where Accountid=%s and cusid=%s""",tuplev)
            conn.execute(*sql)
            return render_template('dash.html',welcome=session.get('username'))
        except:
            error="process coudnt b done"
            return render_template('dash.html',welcome=session.get('username'),val=False)
            print("error")








In [12]:

@app.route('/searchcustomer',methods=['POST','GET'])
def searchcustomer():
     if request.method == 'POST':
        try:
            ssn=request.form['ssnid']
            cid=request.form['cid']
            session['cid'],session['cssn']=cid,ssn
            print(session.get('cid'))
            return render_template('dash.html',welcome=session.get('username'),val=False)
                              
        except:
            error="process coudnt b done"
            return render_template('dash.html',welcome=session.get('username'),val=False)
            print("error")





In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jun/2020 18:46:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2020 18:46:48] "POST /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2020 18:46:48] "GET /static/img/c1.jpeg HTTP/1.1" 404 -


 
123456789
